## Labs 3 and 4
### Data Engineering 1

60 points

In this lab, we are going to build the Country Analysis Relational DataBase (which we will call the C.A.R.D.B. or the "Cardi B"):

![CardbiB](https://media.giphy.com/media/3oEjI5ry4IwZ3RDw6k/giphy.gif "cardib")

We will be collecting data from two sources. First, we will use open data from the World Bank's [Sovereign
Environmental, Social, and Governance (ESG) Data](https://datatopics.worldbank.org/esg/) project. The ESG data reports data from every country in the world over the time frame from 1960-2021 on a wide variety of topics including education, health, and economic factors within the countries. Second, we will use data on the quality and democratic character of countries' governments as reported by the [Varieties of Democracy (V-Dem)](https://www.v-dem.net/data/the-v-dem-dataset/) project at the University of Notre Dame. By using both data sources, we can conduct analyses to see whether democratic openness leads to better societal outcomes for countries. We can also write SQL queries to capture a wide range of information on countries' political parties, tax systems, and banking industries, for example. Or as Cardi B would say, "You in the club just to party, I'm there, I get paid a fee. I be in and out them banks so much, I know they're tired of me."

Our goals are to download the data and load them into our Python environment, to use `pandas` to clean the data and merge corresponding datasets, to confirm that the tables comprise a 3rd normal form relational database, to load the dataframes as entities in a PostgreSQL database, to construct an ER diagram for the database, and to write several SQL queries for the database. 

(1) Initialize a new project for this lab by following these steps:

a. Create a new GitHub repository named "Lab3and4". Choose the option to include a README.md file, select the MIT license and choose a Python .gitignore file. Once the repository is created, edit the .gitignore file on the GitHub website, and add the following lines to the file:

This ensures that none of the data files are uploaded to your public GitHub repo (although the data are open, they have more restrictive licenses, and the best approach is to share code but not the data). [1 point]

b. Use the `git clone` command to download the GitHub files into a folder on your computer. Then change the directory to be inside your local Lab3and4 folder. Next create four new files:

  * A requirements.txt file that loads jupyterlab==3.4.7, requests==2.28.1, pandas==1.5.1, numpy==1.23.4, psycopg2==2.9.5, and sqlalchemy==1.4.42 
    
  * A Dockerfile that loads a recent version of Python with the bulleye installation of Linux, copies the requirements.txt file into the image, runs commands to update pip and install the packages in requirements.txt, runs the commands to install nodejs and dbdocs, sets the working directory to /lab, exposes port 8888, and launches Jupyter lab. Use docker build to create an image from this Dockerfile.
    
  * A .env file that sets your POSTGRES_PASSWORD
    
  * A compose.yaml file that: 
  
    * Runs the postgres:latest image from Docker hub and supplies the .env file and maps port 5432 to your local 5432 port. 
    
    * Runs the image you just created, supplying your .env file, mapping the 8888 port to a port on your comouter, and mapping the /lab directory to your local current working directory. 
    
    * Defines a volume called 'postgresdata' and makes sure the postgres service maps the container's /var/lib/postgresql/data directory to this volume. 
    
    * And defines a network named 'lab3network' and set each service to use this network.

Once you've created these files and run the `docker build` command, type `docker compose up` to launch your project environment. [2 points]

(2) The following problems are a walk-through of the various data wrangling steps we need to take with the data.

Please refer to chapter 8 of "[Surfing the Data Pipeline with Python](https://jkropko.github.io/surfing-the-data-pipeline/ch8.html)" for guidance and examples on how to do these tasks.

a. Import the `pandas`, `numpy`, `requests`, `os`, `psycopg2`, `zipfile`, and `io` packages. (`os`, `zipfile`, and `io` are included in base Python and do not need to be included in your requirements.txt file). Also load the `create_engine` function from `sqlalchemy`. Then load your POSTGRES_PASSWORD environmental variable into your Python kernel. [2 points]

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import requests
import psycopg2
import zipfile
import io
from sqlalchemy import create_engine

postgres_password = os.environ['POSTGRES_PASSWORD']

b. Both the World Bank and V-Dem store their data in zipped directories containing CSV files. Download the World Bank data into your container's top-level directory by typing the following code:

And download the V-Dem data by typing:

There are multiple files downloaded, but only three we need for this project. Load the 'Country_Year_V-Dem_Core_CSV_v12/V-Dem-CY-Core-v12.csv', 'ESGCountry.csv', and 'ESGData.csv' files into your Python kernel. [2 points]

In [2]:
vdem = pd.read_csv('Country_Year_V-Dem_Core_CSV_v12/V-Dem-CY-Core-v12.csv')
countrydata = pd.read_csv('ESGCountry.csv')
wb = pd.read_csv('ESGData.csv')

c. First, let's focus on the V-Dem data ('Country_Year_V-Dem_Core_CSV_v12/V-Dem-CY-Core-v12.csv'). Use `pandas` methods to perform the following tasks:

* Keep only the 'country_text_id', 'country_name','year', 'v2x_polyarchy' columns.

* Use the `.query()` method to keep only the rows in which year is greater than or equal to 1960 and less than or equal to 2021.

* Rename 'country_text_id' to 'country_code', 'country_name' to 'country_name_vdem', and 'v2x_polyarchy' to 'democracy'.

* Sort the rows by 'country_code' and 'year' in ascending order. [2 points]

In [3]:
vdem = vdem[['country_text_id', 'country_name','year', 'v2x_polyarchy']]
vdem = vdem.query("year >= 1960 & year <= 2021")
vdem = vdem.rename({'country_text_id':'country_code',
                    'country_name': 'country_name_vdem',
                   'v2x_polyarchy':'democracy'}, axis=1)
vdem = vdem.sort_values(['country_code', 'year'], ascending=True)
vdem

,country_code,country_name_vdem,year,democracy
5404,AFG,Afghanistan,1960,0.081
5405,AFG,Afghanistan,1961,0.083
5406,AFG,Afghanistan,1962,0.083
5407,AFG,Afghanistan,1963,0.086
5408,AFG,Afghanistan,1964,0.098
...,...,...,...,...
25976,ZZB,Zanzibar,2017,0.259
25977,ZZB,Zanzibar,2018,0.257
25978,ZZB,Zanzibar,2019,0.254
25979,ZZB,Zanzibar,2020,0.251


d. Next let's focus on the country-level World Bank data ('ESGCountry.csv'). Use `pandas` methods to perform the following tasks:

* Keep only the 'Country Code', 'Table Name', 'Long Name', 'Currency Unit', 'Region', and 'Income Group' columns.

* Rename 'Country Code' to 'country_code', 'Table Name' to 'country_name_wb', 'Long Name' to 'country_longname', 'Currency Unit' to 'currency_unit', 'Region' to 'region', and 'Income Group' to 'income_group'.

* The countries in this dataset include various grouping of countries such as "Europe & Central Asia". We want to remove these observations from the data. Use the .query() method to remove the rows in which 'country_name_wb' is equal to one of these non-countries. (Hint: within the .query() method, you can reference an external Python variable such as `noncountries` by placing an @ sign in front of the variable name.) Please use this list of non-countries:

In [4]:
noncountries = ["Arab World", "Central Europe and the Baltics","Caribbean small states",
                "East Asia & Pacific (excluding high income)","Early-demographic dividend","East Asia & Pacific",
                "Europe & Central Asia (excluding high income)","Europe & Central Asia", "Euro area",
                "European Union","Fragile and conflict affected situations","High income",
                "Heavily indebted poor countries (HIPC)","IBRD only","IDA & IBRD total",
                "IDA total","IDA blend","IDA only",
                "Latin America & Caribbean (excluding high income)","Latin America & Caribbean","Least developed countries: UN classification",
                "Low income","Lower middle income","Low & middle income",
                "Late-demographic dividend","Middle East & North Africa","Middle income",
                "Middle East & North Africa (excluding high income)","North America","OECD members",
                "Other small states","Pre-demographic dividend","Pacific island small states",
                "Post-demographic dividend","Sub-Saharan Africa (excluding high income)","Sub-Saharan Africa",
                "Small states","East Asia & Pacific (IDA & IBRD)","Europe & Central Asia (IDA & IBRD)",
                "Latin America & Caribbean (IDA & IBRD)","Middle East & North Africa (IDA & IBRD)","South Asia",
                "South Asia (IDA & IBRD)","Sub-Saharan Africa (IDA & IBRD)","Upper middle income", "World"]

[2 points]

In [5]:
countrydata = countrydata[['Country Code', 'Table Name', 'Long Name', 
                           'Currency Unit', 'Region', 'Income Group']]
countrydata = countrydata.rename({'Country Code':'country_code',
                                 'Table Name':'country_name_wb',
                                 'Long Name':'country_longname',
                                 'Currency Unit':'currency_unit',
                                 'Region':'region',
                                 'Income Group':'income_group'}, axis=1)
countrydata = countrydata.query("country_name_wb not in @noncountries")

e. Next focus on the World Bank country-time-series dataset 'ESGData.csv'. Use `pandas` methods to perform the following tasks:

* Keep only the columns named 'Country Code', 'Country Name', and 'Indicator Code', or begin with '19' or '20'. (Don't type in all the years individually. Instead, use code that finds all columns that begin '19' or '20'.)

* Then use the `.drop()` method to delete the column named '2050'.

* Rename 'Country Code' to'country_code', 'Country Name' to 'country_name_wb', and 'Indicator Code' to 'feature'.

* Type `noncountries.remove('World')` to remove 'World' from the `noncountries` list. (We want to keep the total world data for now). Then use the `.query()` method to remove the rows in which 'country_name_wb' is equal to one of the other entries in the `noncountries` list.

* The features in this dataset are given strange and incomprehensible codes such as 'EG.CFT.ACCS.ZS'. Use the `replace_map` dictionary, defined below, to recode all of these values with more descriptive names for each feature.

In [6]:
replace_map = {'AG.LND.AGRI.ZS': 'agricultural_land',
 'AG.LND.FRST.ZS': 'forest_area',
 'AG.PRD.FOOD.XD': 'food_production_index',
 'CC.EST': 'control_of_corruption',
 'EG.CFT.ACCS.ZS': 'access_to_clean_fuels_and_technologies_for_cooking',
 'EG.EGY.PRIM.PP.KD': 'energy_intensity_level_of_primary_energy',
 'EG.ELC.ACCS.ZS': 'access_to_electricity',
 'EG.ELC.COAL.ZS': 'electricity_production_from_coal_sources',
 'EG.ELC.RNEW.ZS': 'renewable_electricity_output',
 'EG.FEC.RNEW.ZS': 'renewable_energy_consumption',
 'EG.IMP.CONS.ZS': 'energy_imports',
 'EG.USE.COMM.FO.ZS': 'fossil_fuel_energy_consumption',
 'EG.USE.PCAP.KG.OE': 'energy_use',
 'EN.ATM.CO2E.PC': 'co2_emissions',
 'EN.ATM.METH.PC': 'methane_emissions',
 'EN.ATM.NOXE.PC': 'nitrous_oxide_emissions',
 'EN.ATM.PM25.MC.M3': 'pm2_5_air_pollution',
 'EN.CLC.CDDY.XD': 'cooling_degree_days',
 'EN.CLC.GHGR.MT.CE': 'ghg_net_emissions',
 'EN.CLC.HEAT.XD': 'heat_index_35',
 'EN.CLC.MDAT.ZS': 'droughts',
 'EN.CLC.PRCP.XD': 'maximum_5-day_rainfall',
 'EN.CLC.SPEI.XD': 'mean_drought_index',
 'EN.MAM.THRD.NO': 'mammal_species',
 'EN.POP.DNST': 'population_density',
 'ER.H2O.FWTL.ZS': 'annual_freshwater_withdrawals',
 'ER.PTD.TOTL.ZS': 'terrestrial_and_marine_protected_areas',
 'GB.XPD.RSDV.GD.ZS': 'research_and_development_expenditure',
 'GE.EST': 'government_effectiveness',
 'IC.BUS.EASE.XQ': 'ease_of_doing_business_rank',
 'IC.LGL.CRED.XQ': 'strength_of_legal_rights_index',
 'IP.JRN.ARTC.SC': 'scientific_and_technical_journal_articles',
 'IP.PAT.RESD': 'patent_applications',
 'IT.NET.USER.ZS': 'individuals_using_the_internet',
 'NV.AGR.TOTL.ZS': 'agriculture',
 'NY.ADJ.DFOR.GN.ZS': 'net_forest_depletion',
 'NY.ADJ.DRES.GN.ZS': 'natural_resources_depletion',
 'NY.GDP.MKTP.KD.ZG': 'gdp_growth',
 'PV.EST': 'political_stability_and_absence_of_violence',
 'RL.EST': 'rule_of_law',
 'RQ.EST': 'regulatory_quality',
 'SE.ADT.LITR.ZS': 'literacy_rate',
 'SE.ENR.PRSC.FM.ZS': 'gross_school_enrollment',
 'SE.PRM.ENRR': 'primary_school_enrollment',
 'SE.XPD.TOTL.GB.ZS': 'government_expenditure_on_education',
 'SG.GEN.PARL.ZS': 'proportion_of_seats_held_by_women_in_national_parliaments',
 'SH.DTH.COMM.ZS': 'cause_of_death',
 'SH.DYN.MORT': 'mortality_rate',
 'SH.H2O.SMDW.ZS': 'people_using_safely_managed_drinking_water_services',
 'SH.MED.BEDS.ZS': 'hospital_beds',
 'SH.STA.OWAD.ZS': 'prevalence_of_overweight',
 'SH.STA.SMSS.ZS': 'people_using_safely_managed_sanitation_services',
 'SI.DST.FRST.20': 'income_share_held_by_lowest_20pct',
 'SI.POV.GINI': 'gini_index',
 'SI.POV.NAHC': 'poverty_headcount_ratio_at_national_poverty_lines',
 'SI.SPR.PCAP.ZG': 'annualized_average_growth_rate_in_per_capita_real_survey_mean_consumption_or_income',
 'SL.TLF.0714.ZS': 'children_in_employment',
 'SL.TLF.ACTI.ZS': 'labor_force_participation_rate',
 'SL.TLF.CACT.FM.ZS': 'ratio_of_female_to_male_labor_force_participation_rate',
 'SL.UEM.TOTL.ZS': 'unemployment',
 'SM.POP.NETM': 'net_migration',
 'SN.ITK.DEFC.ZS': 'prevalence_of_undernourishment',
 'SP.DYN.LE00.IN': 'life_expectancy_at_birth',
 'SP.DYN.TFRT.IN': 'fertility_rate',
 'SP.POP.65UP.TO.ZS': 'population_ages_65_and_above',
 'SP.UWT.TFRT': 'unmet_need_for_contraception',
 'VA.EST': 'voice_and_accountability'}

[2 points]

In [7]:
tokeep = [x for x in wb.columns if x.startswith('19') or x.startswith('20')]
wb = wb[['Country Code', 'Country Name', 'Indicator Code'] + tokeep]
wb = wb.drop(['2050'], axis=1)
wb = wb.rename({'Country Code':'country_code',
                'Country Name':'country_name_wb',
               'Indicator Code':'feature'}, axis=1)
noncountries.remove('World')
wb = wb.query("country_name_wb not in @noncountries")
wb['feature'] = wb['feature'].replace(replace_map)

f. The World Bank country-time-series dataset is strangely organized. The features are stored in the rows, when typically we would want these features to be columns. Also, years are stored in columns, when typically we would want years to be represented by different rows. We can repair this structure by reshaping the data. Following the discussion in chapter 9 of "[Surfing the Data Pipeline with Python](https://jkropko.github.io/surfing-the-data-pipeline/ch9.html)", perform the following reshapes:

* First, turn the columns that refer to years into rows,

* Then turn the rows that refer to features into columns. [3 points]

In [8]:
wb = pd.melt(wb, id_vars = ['country_code', 'country_name_wb', 'feature'], value_vars = [str(i) for i in range(1960,2022)],
            var_name = 'year')
wb = wb.pivot_table(index = ['country_code', 'country_name_wb', 'year'], 
                    columns = 'feature', 
                    values = 'value').reset_index()

g. After these reshapes, the year column in the World Bank country-time-series dataset is stored as a string. Convert this column to an integer data type. [1 point]

In [9]:
wb.year = wb.year.astype('int')

h. Create a new dataframe containing just the rows from the World Bank country-time-series dataset that refer to the whole world. Then remove the "World" rows from the World Bank country-time-series dataset. Drop the 'country_code' and 'country_name_wb' from the world dataset. Finally, with the exception of the year column, rename all columns in the world dataset by adding "world_" in front of the column name. [2 points]

In [10]:
world = wb.query("country_name_wb == 'World'")
wb = wb.query("country_name_wb != 'World'")
world = world.drop(['country_code', 'country_name_wb'], axis=1)
world.columns = ['world_' + c for c in world.columns]
world = world.rename({'world_year':'year'}, axis=1)

i. Next we will merge the World Bank country-time-series dataset with the V-Dem dataset, matching on the 'country_code' and 'year' columns. First, write a sentence stating whether you expect this merge to be one-to-one, many-to-one, one-to-many, or many-to-many, and describe your rationale. Second, merge the two datasets together in a way that checks whether your expectation is met, and also allows you to see the rows that failed to match. [3 points]

I expect the merge to be one-to-one because country_code and year are a primary key in both the V-Dem and World Bank datasets: no two rows should have the same value of both country_code and year.

In [11]:
country_series = pd.merge(wb, vdem,
                         on = ['country_code', 'year'],
                         how = 'outer',
                         validate = 'one_to_one',
                         indicator = 'matched')

j. After this merge, use the `.value_counts()` method to see the total numnber of observations that were found in both datasets, the number found only in the left dataset, and the number found only in the right dataset. (If you entered the World Bank dataset into the merge function first, then "left_only" refers to the rows found in the World Bank but not V-Dem, and "right_only refers to the rows found in V-Dem but not the World Bank.) There should be more than 9000 rows that matched, but more than 2000 that failed to match.

Then conduct two data aggregations to help us investigate why these observations did not match:

* First use `.query()` to keep only the observations that were present in the World Bank data but not V-Dem. (These are the 'left_only' observations if you typed the World Bank data into the merge function first.) Use `.groupby()` to aggregate the data by 'country_code' and 'country_name_wb'. Then save the minimum and maximum values of 'year' for each country.

* Then use `.query()` to keep only the observations that were present in the V-Dem data but not the World Bank. Use `.groupby()` to aggregate the data by 'country_code' and 'country_name_vdem'. Then save the minimum and maximum values of 'year' for each country. [3 points]

In [12]:
country_series.matched.value_counts()

both          9976
left_only     1984
right_only     395
Name: matched, dtype: int64

In [13]:
country_series.query("matched == 'left_only'").groupby(['country_code', 'country_name_wb']).agg({'year':[min, max]})

year      
                                              min   max
country_code country_name_wb                           
AND          Andorra                         1961  2021
ARE          United Arab Emirates            1960  1970
ARM          Armenia                         1960  1989
ATG          Antigua and Barbuda             1960  2021
AZE          Azerbaijan                      1960  1989
BGD          Bangladesh                      1960  1970
BHS          Bahamas, The                    1960  2021
BIH          Bosnia and Herzegovina          1960  1991
BLR          Belarus                         1960  1989
BLZ          Belize                          1960  2021
BRN          Brunei Darussalam               1960  2021
CMR          Cameroon                        1960  1960
DMA          Dominica                        1960  2021
EST          Estonia                         1960  1989
FSM          Micronesia, Fed. Sts.           1960  2021
GEO          Georgia                         1960  1989
GRD          Grenada                         1960  2021
HRV          Croatia                         1960  1990
KAZ          Kazakhstan                      1960  1989
KGZ          Kyrgyz Republic                 1960  1989
KIR          Kiribati                        1960  2021
KNA          St. Kitts and Nevis             1960  2021
LCA          St. Lucia                       1960  2021
LIE          Liechtenstein                   1961  2021
LTU          Lithuania                       1960  1989
LVA          Latvia                          1960  1989
MCO          Monaco                          1961  2021
MDA          Moldova                         1960  1989
MHL          Marshall Islands                1960  2021
MKD          North Macedonia                 1960  1990
MNE          Montenegro                      1960  1997
NRU          Nauru                           1961  2021
PLW          Palau                           1961  2021
SMR          San Marino                      1961  2021
SSD          South Sudan                     1960  2010
SVK          Slovak Republic                 1960  1992
SVN          Slovenia                        1960  1988
TJK          Tajikistan                      1960  1989
TKM          Turkmenistan                    1960  1989
TON          Tonga                           1960  2021
TUV          Tuvalu                          1960  2021
UKR          Ukraine                         1960  1989
UZB          Uzbekistan                      1960  1989
VCT          St. Vincent and the Grenadines  1960  2021
WSM          Samoa                           1960  2021

In [14]:
country_series.query("matched == 'right_only'").groupby(['country_code', 'country_name_vdem']).agg({'year':[min, max]})

year      
                                          min   max
country_code country_name_vdem                     
DDR          German Democratic Republic  1960  1990
HKG          Hong Kong                   1960  2021
PSE          Palestine/West Bank         1967  2021
PSG          Palestine/Gaza              1960  2021
SML          Somaliland                  1991  2021
TWN          Taiwan                      1960  2021
VDR          Republic of Vietnam         1960  1975
XKX          Kosovo                      1999  2021
YMD          South Yemen                 1960  1990
ZZB          Zanzibar                    1960  2021

k. Here's where a deep understanding of the data becomes very important. There are two reasons why an observation may fail to match in a merge. One reason is a difference in spelling. Suppose that South Korea (which is also known as the Republic of Korea) is coded as SKO in the World Bank data and ROK in V-Dem. In this case, we should recode one or the other of SKO and ROK so that they match, otherwise we will lose the data on South Korea. But the second reason why observations might fail to match is due to differences in coverage in the data collection strategy: it is possible that a country wasn't included in one data's coverage, or that certain years for that country were not included. For differences in coverage, there's no way to manipulate the data to match, so we are out of luck and we have to either delete these observations or proceed with missing data from one of the data sources.

Take a close look at the two data aggregation tables you generated in part (j), and answer the following questions:

* Do you see any countries that are present in both the unmatched World Bank rows and the unmatched V-Dem rows, but with different spellings?

* Do some digging on Wikipedia and other sources on the Internet. What do you think is the primary reason why some countries are present in the V-Dem data but not the World Bank? (You don't need to describe the reasoning for every country. Just dig until you see a general pattern and describe it here.)

* Do some more digging on Wikipedia and other sources on the Internet. What do you think is the primary reason why some countries are present in the World Bank data but not V-Dem? (You don't need to describe the reasoning for every country. Just dig until you see a general pattern and describe it here.) [3 points]

* No, there are no differences in spelling.

* The primary reason why countries are included in V-Dem but not the World Bank is that these countries are ones that no longer exist (such as the GDR, the Republic of Vietnam, and South Yemen), or whose sovereignty is contested or not officially recognized by the United Nations (Hong Kong, Palestine, Somaliland, Taiwan, Kosovo, Zanzibar).

* The primary reason why countries are included in the World Bank data but not V-Dem is that these are countries which achieved independence and internationally-recognized sovereignty at some point during the 1960-2021 time frame, such as former Soviet countries (Latvia, Ukraine, Estonia, etc.). The World Bank started collecting statistics on these countries before they were independent.

l. Once you are convinced that all of the unmatched observations are due to differences in the coverage of the data collection strategies of the World Bank and V-Dem, repeat the merge, this time dropping all unmatched observations. This time there is no need to validate the type of merge, and no need to define a variable to indicate matching. [1 point]

(3) After working through all of the data wrangling steps in problem 2, you now have the following data frames:

* The World Bank's country level indicators (we'll call this table "country")

* The World Bank's indicators for the whole world through the 1960-2021 timeframe (we'll call this table "world")

* The merged data frame containing both the World Bank's and V-Dem's country-time-series data (we'll call this table "timeseries")

Do these three tables together comprise a third normal form database? Write a paragraph that describes your reasoning, and be clear about whether the criteria or 1st, 2nd, and 3rd normal forms have been achieved. If the data are not in 3rd normal form, take steps to put the data in 3rd normal form. (Hint: it's not important to keep the country names from both the World Bank and V-Dem. If one of these names is stored in one of the tables, that works.) [4 points]

(4) Connect to the PostgreSQL server that is running on port 5432. Create a new database called 'cardib'. Then use `sqlalchemy` to create an engine that connects to this database, and upload the country, world, and timeseries tables to this database. (Hint: set 'user' in the `psycopg2.connect()` method to be 'postgres', and set 'host' to be whatever you named the PostgreSQL service in your compose.yaml file. I named this service 'postgres' in my file. In addition, change 'localhost' in the `create_engine()` function to the name of this service as well.) [4 points]

(5) Use dbdocs to create an ER diagram for this database. Copy the text of your .dmbl file and paste it in a "raw" type cell below. Then copy the URL of the published ER diagram and paste it in a markdown cell.

A few points to keep in mind:

* In the timeseries table, country_code is varchar, year is int, and every feature is float.

* Pay close attention to whether the tables have a one-to-one, many-to-one, or one-to-many relationship with each other, and make sure the ER diagram properly represents that relationship.

* It would be good to write a sentence or two describing the sources of the data as a note for the database. [5 points]

(6) Write SQL queries for the cardib database that perform the following tasks, and display the output: [3 points each]

a. *What countries had the highest quality democracies in the year 2021?* Join the timeseries and country tables, select the country_name_wb and democracy attributes, rename county_name_wb to country, filter the rows to only those for the year 2021, and sort the rows in descending order by the value of democracy.

b. *How does the life expectancy at birth for Chile compare to the global average life expectancy at birth over the 1960-2021 time span?* Join the timeseries and world tables, select year from the timeseries table and the life expectancy attributes from both the timeseries and world tables, filter the rows to just those where the country code is CHL, and sort the rows by year.

c. *What regions of the world generated the most carbon dioxide emissions in 2019?* Join the timeseries and country tables, filter the rows to only the ones that refer to 2019, and aggregate the data by region. Keep the region column from the country table and the sum of the co2_emissions attribute from the timeseries table. Rename the sum to co2_emissions. Then sort the rows in descending order by co2_emissions.

d. *What countries experienced the greatest increases in democratic quality over the 1960-2021 time span?* Use subqueries to create two new tables on the fly. One table contains country_code and democracy from the timeseries table in which only the rows from 1960 are present. The second table contains country_code and democracy from the timeseries table in which only the rows from 2021 are present. Join these two tables together, then join the result with the country table. Select the country name from county, democracy from the 1960 table, democracy from the 2021 table, and the difference between the two democracy scores. Give each column a more intuitive name. Then sort the rows in descending order by the democracy difference and display the first 10 rows.

For the next two questions, write a query that answers the given question without additional guidance.

e. *By count of countries, what is the most commonly used currency in the world?*

f. *The GINI index measures the amount of economic inequality in a country. The higher the index, the greater the economic disparity between rich and poor. How does the average GINI index compare across income groups (see the country table) in 2019?* 

Extra credit: *Some countries include the word 'Republic' or 'Democratic' in their official names. Other countries do not. Which of these two groups of countries had a higher average level of democratic quality in 2021?* (It's OK if you use two queries to provide this answer.) [5 bonus points]